<a href="https://colab.research.google.com/github/veloni-alt/BioInfo/blob/main/projeto_bioinformaticav2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests, sys
rsid=input('Inclua o RSID de pesquisa:')
link = f"https://rest.ensembl.org/variation/human/{rsid}"
headers = {"Content-Type": "application/json"}
r = requests.get(link, headers=headers)

print(r.json())



Clique no link abaixo para abrir o seu site:
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit



Welcome to localhost.run!

Follow your favourite reverse tunnel at [https://twitter.com/localhost_run].

To set up and manage custom domains go to https://admin.localhost.run/

More details on custom domains (and how to enable subdomains of your custom
domain) at https://localhost.run/docs/custom-domains

If you get a permission denied error check the faq for how to connect with a key or
create a free tunnel without a key at [http://localhost:3000/docs/faq#generating-an-ssh-key].

To explore using localhost.run visit the documentation site:
https://localhost.run/docs/


** your connection id is 8cd1d6bc-627f-47ef-b9a1-a8d2d2cd5b58, please mention it if you send me a message about an issue. **

authenticated as anonymous user
33356efb8bd8ae.lhr.life tunneled with tls termination, https://33356efb8bd8ae.lhr.life
create an account and add your key for a longer lasting domain name. see https://localhost.run/docs/forever-free/ for more information.
Open your tunnel address on your mobile w

INFO:werkzeug:127.0.0.1 - - [28/Dec/2025 23:45:56] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Dec/2025 23:45:57] "GET /favicon.ico HTTP/1.1" 404 -
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unex

In [2]:
from flask import Flask, render_template_string, request
from threading import Thread
import requests
import os

app = Flask(__name__)

# HTML com estilo para a Tabela
html_layout = '''
<!DOCTYPE html>
<html>
<head>
    <title>BioInfo Table</title>
    <style>
        body { font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; margin: 40px; background-color: #e3f2fd; }
        .container { background: white; padding: 30px; border-radius: 12px; box-shadow: 0 4px 15px rgba(0,0,0,0.1); max-width: 800px; margin: auto; }
        h2 { color: #2c3e50; text-align: center; }
        .search-box { display: flex; gap: 10px; margin-bottom: 30px; justify-content: center; }
        input { padding: 12px; width: 60%; border: 2px solid #e0e0e0; border-radius: 6px; outline: none; transition: border-color 0.3s; }
        input:focus { border-color: #007bff; }
        button { padding: 12px 25px; background-color: #007bff; color: white; border: none; border-radius: 6px; cursor: pointer; font-weight: bold; }
        button:hover { background-color: #0056b3; }

        table { width: 100%; border-collapse: collapse; margin-top: 20px; background: white; }
        th, td { padding: 12px; text-align: left; border-bottom: 1px solid #ddd; }
        th { background-color: #f2f2f2; color: #333; text-transform: uppercase; font-size: 12px; }
        tr:hover { background-color: #f9f9f9; }
        .error { color: #d9534f; background: #fdf7f7; padding: 15px; border-radius: 6px; border: 1px solid #eed3d7; }
    </style>
</head>
<body>
    <div class="container">
        <h2>🧬 Explorador Genético (RSID)</h2>
        <div class="search-box">
            <form method="POST" style="width: 100%; display: flex; justify-content: center; gap: 10px;">
                <input type="text" name="rsid_input" placeholder="Ex: rs53576" required>
                <button type="submit">Analisar</button>
            </form>
        </div>

        {% if erro %}
            <div class="error">{{ erro }}</div>
        {% endif %}

        {% if dados %}
            <h3>Dados da Variante: {{ rsid }}</h3>
            <table>
                <thead>
                    <tr>
                        <th>Propriedade</th>
                        <th>Valor</th>
                    </tr>
                </thead>
                <tbody>
                    {% for chave, valor in dados.items() %}
                    <tr>
                        <td><strong>{{ chave }}</strong></td>
                        <td>{{ valor | string }}</td>
                    </tr>
                    {% endfor %}
                </tbody>
            </table>
        {% endif %}
    </div>
</body>
</html>
'''

@app.route('/', methods=['GET', 'POST'])
def home():
    dados_formatados = None
    erro_msg = None
    rsid_solicitado = None

    if request.method == 'POST':
        rsid_solicitado = request.form.get('rsid_input').strip()
        url = f"https://rest.ensembl.org/variation/human/{rsid_solicitado}"
        headers = {"Content-Type": "application/json"}

        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                # Pegamos o JSON e passamos para o template como um dicionário
                dados_formatados = response.json()
            else:
                erro_msg = f"Variante '{rsid_solicitado}' não encontrada na base Ensembl."
        except Exception as e:
            erro_msg = f"Erro de conexão: {str(e)}"

    return render_template_string(html_layout, dados=dados_formatados, erro=erro_msg, rsid=rsid_solicitado)

def run_app():
    app.run(port=5000, host='0.0.0.0', debug=False)

# Inicia o servidor em Thread
Thread(target=run_app).start()

# Gera o link de acesso público
print("Clique no link abaixo para abrir o seu site:")
!ssh -o StrictHostKeyChecking=no -R 80:localhost:5000 nokey@localhost.run

 * Serving Flask app '__main__'
Clique no link abaixo para abrir o seu site:
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.



Welcome to localhost.run!

Follow your favourite reverse tunnel at [https://twitter.com/localhost_run].

To set up and manage custom domains go to https://admin.localhost.run/

More details on custom domains (and how to enable subdomains of your custom
domain) at https://localhost.run/docs/custom-domains

If you get a permission denied error check the faq for how to connect with a key or
create a free tunnel without a key at [http://localhost:3000/docs/faq#generating-an-ssh-key].

To explore using localhost.run visit the documentation site:
https://localhost.run/docs/


** your connection id is 0db22de2-bae7-4b65-b46d-7cc91c0bf83b, please mention it if you send me a message about an issue. **

authenticated as anonymous user
bff18a11f6da33.lhr.life tunneled with tls termination, https://bff18a11f6da33.lhr.life
create an account and add your key for a longer lasting domain name. see https://localhost.run/docs/forever-free/ for more information.
Open your tunnel address on your mobile w

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_debugpy_repr.py", line 5